## Detailed Analysis
### GAIA Star Cluster Member Classification    
#### DATA 301, 2023

**Sakila Wanigasinghe [300526406]**

***

In [21]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier, plot_importance
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve, precision_score, balanced_accuracy_score, recall_score, f1_score, plot_roc_curve
from sklearn.preprocessing import LabelEncoder

<br>

**Loading in data for star clusters: *NGC 7789*, and *Trumpler 5***

In [22]:
## Loading in NGC 7789 star cluster data
ngc_7789 = pd.read_csv('../Data/ngc_7789.csv')
ngc_7789.shape

(3575, 13)

In [23]:
ngc_7789.head()

,source_id,ra,dec,parallax,pmra,pmdec,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,radial_velocity,teff_gspphot,Cluster
0,420901507912889088,0.082375,56.014056,0.320644,-0.386428,-1.528307,16.820808,17.286600,16.184370,1.102230,NaN,5409.3820,Other
1,420906936751548288,0.237080,56.055887,0.597838,-1.651632,-2.148538,17.591908,18.355732,16.755722,1.600010,NaN,4343.0300,Other
2,420907142909967104,0.149607,56.061129,0.211846,-1.312115,-2.421624,15.102909,15.690556,14.344952,1.345604,NaN,5978.2773,Other
3,420907722722130816,0.003523,56.075048,0.342945,-1.708469,-2.459792,16.090326,16.448742,15.559808,0.888934,NaN,6960.6714,Other
4,420908414210912512,0.013300,56.152271,0.560820,-1.245880,-2.100259,16.771912,NaN,NaN,NaN,NaN,NaN,Other


In [24]:
## Loading in Trumpler 5 star cluster data
trumpler_5 = pd.read_csv('../Data/trumpler_5.csv')
trumpler_5.shape

(3302, 13)

In [25]:
trumpler_5.head()

,source_id,ra,dec,parallax,pmra,pmdec,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,radial_velocity,teff_gspphot,Cluster
0,3326188665921045120,99.180263,8.938090,0.117289,-0.767038,0.202383,17.720343,18.314255,16.970951,1.343304,NaN,5201.7600,Other
1,3326188940798945920,99.123816,8.924983,-0.052569,0.351594,-0.838942,17.792180,18.782574,16.812962,1.969612,NaN,4615.8970,Other
2,3326189391771063424,99.161032,8.937259,0.354193,-0.229926,-0.055456,16.855852,17.416264,16.147139,1.269125,NaN,5466.8804,Other
3,3326189421835294080,99.175908,8.955257,0.423364,0.278428,-0.069258,17.581598,18.265062,16.802801,1.462261,NaN,5164.1900,Other
4,3326189593633986432,99.104969,8.958145,0.333585,-0.597968,-0.295040,17.811178,18.439910,17.031984,1.407927,NaN,4466.0156,Other


<br>

### NGC 7789 Star Cluster Members Classification

**Creating train, validation, and test sets**

In [26]:
ngc_7789_filtered = ngc_7789.copy()

In [27]:
## Removing redundant column 'source_id' and 'radial_velocity'
ngc_7789_filtered = ngc_7789[ngc_7789['ra'] > 350]
ngc_7789_filtered = ngc_7789_filtered.drop(['radial_velocity', 'source_id'], axis = 1)
ngc_7789_filtered.shape

(3311, 11)

In [28]:
ngc_7789_filtered.head()

,ra,dec,parallax,pmra,pmdec,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,teff_gspphot,Cluster
264,359.973766,55.980510,0.345684,-1.264863,-2.185392,17.272950,17.700705,16.653336,1.047369,5222.2837,Other
265,359.800712,55.948081,0.476470,-0.785729,-1.906318,17.525180,18.180155,16.755987,1.424168,4296.6250,NGC_7789
266,359.931384,56.017455,0.454876,-1.166511,-1.637231,17.030367,17.451424,16.395910,1.055513,5608.1465,Other
267,359.458759,55.896147,0.480959,-1.395051,-1.462129,17.948677,18.468685,17.260397,1.208288,4736.8400,Other
268,359.498434,55.904656,0.258754,-0.458683,-2.072979,17.734300,18.214066,17.100204,1.113861,4929.7090,Other


In [29]:
## Creating train and test sets
ngc_train_val, ngc_test = train_test_split(ngc_7789_filtered, test_size = 0.20, random_state = 309)
print(ngc_train_val.shape, ngc_test.shape)

(2648, 11) (663, 11)


In [30]:
## Creating validation set from training set
ngc_train, ngc_val = train_test_split(ngc_train_val, test_size = 0.15, random_state = 309)
print(ngc_train.shape, ngc_val.shape)

(2250, 11) (398, 11)


In [31]:
## Splitting train, test, and validation sets into corresponding X and y
target = 'Cluster'
ngc_X_train = ngc_train.drop(target, axis = 1) 
ngc_y_train = ngc_train[target].copy() 

ngc_X_val = ngc_val.drop(target, axis = 1) 
ngc_y_val = ngc_val[target].copy() 

ngc_X_test = ngc_test.drop(target, axis = 1) 
ngc_y_test = ngc_test[target].copy() 

print(ngc_X_train.shape, ngc_y_train.shape, ngc_X_val.shape, ngc_y_val.shape, ngc_X_test.shape, ngc_y_test.shape)

(2250, 10) (2250,) (398, 10) (398,) (663, 10) (663,)


**Model Fitting and Preprocessing Function (Pipeline)**

In [32]:
def ngc_xgb_transform(X, y):
    '''XGBoost preprocessing and model fitting custom function for NGC 7789 data'''
    
    numerical_columns = ["ra", "dec", "parallax", "pmra", "pmdec", "phot_g_mean_mag", "phot_bp_mean_mag", "phot_rp_mean_mag", "bp_rp", "teff_gspphot"]

    numerical_preprocessor = Pipeline([('scaler', StandardScaler()),
                                       ('imputer', SimpleImputer(strategy = 'median'))])

    preprocessor = ColumnTransformer([
        ('scaler-and-imputer', numerical_preprocessor, numerical_columns),
    ])

    xgb_label_encoder = LabelEncoder()
    y = xgb_label_encoder.fit_transform(y)

    xgb_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', XGBClassifier(objective = 'binary:logistic', random_state = 309))
    ])

    model = xgb_pipeline.fit(X, y)

    return(model, xgb_label_encoder)

**Model Evaluation Function**

In [33]:
## Defining custom function to evaluate model performance
def model_performance(model, X, y, label_enc):
    '''Generates performance metrics for the model, also can produce graphs of the model performance'''

    y_predicted = model.predict(X)

    y = label_enc.transform(y)
    y_predicted = model.predict(X) 
    y_predicted = label_enc.inverse_transform(y_predicted)
    
    cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 1, random_state = 309)
    
    scores_RA = cross_val_score(model, X, y, cv = cv, scoring = "roc_auc")
    scores_BA = cross_val_score(model, X, y, cv = cv, scoring = "balanced_accuracy")    
        
    print(f"AUC-ROC: {scores_RA.mean():.4f} +/- {scores_RA.std():.4f}")
    print(f"Balanced accuracy score: {scores_BA.mean():.4f} +/- {scores_BA.std():.4f}")

**XGBoost model training and evaluation**

In [34]:
## Performance of model on training data
ngc_xgb_model = ngc_xgb_transform(ngc_X_train, ngc_y_train)
model_performance(ngc_xgb_model[0], ngc_X_train, ngc_y_train, label_enc = ngc_xgb_model[1])

AUC-ROC: 0.9001 +/- 0.0109
Balanced accuracy score: 0.8153 +/- 0.0214


In [35]:
## Performance of model on validation data
ngc_xgb_model = ngc_xgb_transform(ngc_X_val, ngc_y_val)
model_performance(ngc_xgb_model[0], ngc_X_val, ngc_y_val, label_enc = ngc_xgb_model[1])

AUC-ROC: 0.8903 +/- 0.0491
Balanced accuracy score: 0.8125 +/- 0.0750


In [39]:
## Fitting model, removing correlated features
def ngc_xgb_transform_2(X, y):
    '''XGBoost preprocessing and model fitting custom function for NGC 7789 data'''
    
    numerical_columns = ["ra", "dec", "parallax", "pmra", "pmdec", "phot_g_mean_mag"]

    numerical_preprocessor = Pipeline([('scaler', StandardScaler()),
                                       ('imputer', SimpleImputer(strategy = 'median'))])

    preprocessor = ColumnTransformer([
        ('scaler-and-imputer', numerical_preprocessor, numerical_columns),
    ])

    xgb_label_encoder = LabelEncoder()
    y = xgb_label_encoder.fit_transform(y)

    xgb_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', XGBClassifier(objective = 'binary:logistic', random_state = 309))
    ])

    model = xgb_pipeline.fit(X, y)

    return(model, xgb_label_encoder)

In [37]:
ngc_X_train = ngc_X_train.drop(["phot_bp_mean_mag", "phot_rp_mean_mag", "bp_rp", "teff_gspphot"], axis = 1)
ngc_X_val = ngc_X_val.drop(["phot_bp_mean_mag", "phot_rp_mean_mag", "bp_rp", "teff_gspphot"], axis = 1)

In [40]:
## Performance of model on training data
ngc_xgb_model = ngc_xgb_transform_2(ngc_X_train, ngc_y_train)
model_performance(ngc_xgb_model[0], ngc_X_train, ngc_y_train, label_enc = ngc_xgb_model[1])

AUC-ROC: 0.8966 +/- 0.0075
Balanced accuracy score: 0.8120 +/- 0.0194


In [41]:
## Performance of model on validation data
ngc_xgb_model = ngc_xgb_transform_2(ngc_X_val, ngc_y_val)
model_performance(ngc_xgb_model[0], ngc_X_val, ngc_y_val, label_enc = ngc_xgb_model[1])

AUC-ROC: 0.8936 +/- 0.0550
Balanced accuracy score: 0.8206 +/- 0.0692


<br>

***

### Trumpler 5 Star Cluster Members Classification

In [13]:
def xgb_transform_trumpler5(X, y, synthetic_sampling = True):
    '''XGBoost preprocessing and model fitting custom function'''
    
    numerical_columns = ["ra", "dec", "parallax", "pmra", "pmdec", "phot_g_mean_mag", "phot_bp_mean_mag", "phot_rp_mean_mag", "bp_rp", "teff_gspphot"]

    numerical_preprocessor = Pipeline([('scaler', StandardScaler()),
                                       ('imputer', SimpleImputer(strategy = 'median'))])

    preprocessor = ColumnTransformer([
        ('scaler-and-imputer', numerical_preprocessor, numerical_columns),
    ])

    xgb_label_encoder = LabelEncoder()
    y = xgb_label_encoder.fit_transform(y)


    if synthetic_sampling == True:
        xgb_pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ("over", SMOTE(sampling_strategy = 0.80, random_state = 309)),
            ("under", RandomUnderSampler(sampling_strategy = 1.0, random_state = 309)),
            ('classifier', XGBClassifier(objective = 'binary:logistic', random_state = 309))
        ])

    else:
        xgb_pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', XGBClassifier(objective = 'binary:logistic', random_state = 309))
        ])

    model = xgb_pipeline.fit(X, y)

    return(model, xgb_label_encoder)